In [1]:
'''
在2014年，GoogLeNet赢得了ImageNet挑战赛（Szegedy等人，2015），它使用了一种结构，结合了NiN（Lin等人，2013）的优点、重复的卷积块（Simonyan和Zisserman，2014）以及各种卷积核的组合。可以说，它也是第一个在CNN中明确展示出茎（数据摄取）、主体（数据处理）和头部（预测）区分的网络。此设计模式自那时以来一直在深度网络设计中保持不变：茎由前两个或三个对图像进行操作的卷积组成。它们从底层图像中提取低级特征。接下来是一系列卷积块组成的主体。最后，头部将迄今为止获得的特征映射到所需的分类、分割、检测或跟踪问题。

GoogLeNet的关键贡献在于网络主体的设计。它以一种巧妙的方式解决了选择卷积核的问题。虽然其他工作试图确定哪种卷积，从1*1到11*11之间的卷积核最好，但GoogLeNet只是简单地将多分支卷积连接在一起。在接下来的内容中，我们将介绍GoogLeNet的一个略微简化版：原始设计包括一些技巧，通过对网络的多个层应用中间损失函数来稳定训练。由于现有更好的训练算法，这些技巧不再必要。

GoogLeNet的关键组成部分是Inception模块。Inception模块包括多个并行卷积分支，这些分支具有不同大小的卷积核，如1*1、3*3和5*5。这使得网络能够捕捉不同尺度的特征。此外，Inception模块还包括一个最大池化分支。为了减少计算成本，GoogLeNet在应用较大卷积核之前使用了1*1卷积来减少通道数。

GoogLeNet的网络结构包括一个输入茎，由几个卷积层组成，用于提取低级特征。接下来是一系列Inception模块组成的主体，用于捕捉多尺度特征。最后，网络头部使用全局平均池化和一个全连接层来生成预测结果。

总之，GoogLeNet结合了NiN、重复卷积块和多种卷积核的优点，提出了一种具有创新性的网络结构。通过Inception模块，GoogLeNet能够在保持较低计算成本的同时捕捉多尺度特征，从而在各种计算机视觉任务中取得了显著的成功。
'''
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
class Inception(nn.Module):
    # c1--c4 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

In [4]:
class GoogleNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [5]:
@d2l.add_to_class(GoogleNet)
def b2(self):
    return nn.Sequential(
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [6]:
@d2l.add_to_class(GoogleNet)
def b3(self):
    return nn.Sequential(Inception(64, (96, 128), (16, 32), 32),
                         Inception(128, (128, 192), (32, 96), 64),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [7]:
@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [8]:
@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(Inception(256, (160, 320), (32, 128), 128),
                         Inception(384, (192, 384), (48, 128), 128),
                         nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten())

In [9]:
@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    super(GoogleNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(), self.b4(),
                             self.b5(), nn.LazyLinear(num_classes))
    self.net.apply(d2l.init_cnn)

In [10]:
model = GoogleNet().layer_summary((1, 1, 96, 96))

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


In [11]:
model = GoogleNet(lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
GoogLeNet的一个关键特性是它在计算成本上比前辈们更低，同时提供了更高的准确性。这标志着更加深入的网络设计的开始，权衡评估网络的成本与减少错误的关系。
这同时也标志着在块级别上开始尝试网络设计超参数的实验，尽管当时这一切都是完全手动进行的。在第8.8节中，我们将在讨论网络结构探索策略时重新审视这个主题。

在接下来的几节中，我们将遇到许多设计选择（例如，批量归一化、残差连接和通道分组），这些选择使我们能够显著改进网络。现在，你可以为实现了可以说是第一个真正现代化的CNN而自豪。

GoogLeNet的成功源于其创新的网络设计，特别是Inception模块的引入，这使得网络能够在保持较低计算成本的同时提高准确性。这种设计思路为后续的深度学习研究和应用奠定了基础，促使了更多关于网络结构和优化技术的探索。

随着深度学习领域的发展，我们将看到更多的设计选择，如批量归一化、残差连接和通道分组等，这些技术将进一步提高网络的性能。GoogLeNet的成功为深度学习领域开辟了新的道路，为实现更高性能的现代CNN奠定了基础。
'''

GoogLeNet经过了多次迭代，逐步提高了速度和准确性。尝试实现并运行其中的一些版本，包括以下内容：

1. 添加批量归一化层（Ioffe和Szegedy，2015），稍后在第8.5节中详细描述。
2. 调整Inception块（宽度，卷积的选择和顺序），如Szegedy等人（2016）所述。
3. 使用标签平滑进行模型正则化，如Szegedy等人（2016）所述。
4. 通过添加残差连接（Szegedy等人，2017）进一步调整Inception块，稍后在第8.6节中详细描述。

问题：

1. GoogLeNet需要的最小图像尺寸是多少？

GoogLeNet需要的最小图像尺寸为224x224像素。

2. 你能设计一个适用于Fashion-MNIST原始分辨率28x28像素的GoogLeNet变体吗？你需要如何改变网络的茎、主体和头部？

为了使GoogLeNet适用于Fashion-MNIST的原始分辨率，您需要调整网络的茎、主体和头部。具体来说，您需要减少茎部分的卷积层数量和卷积核大小，以适应较小的输入图像。同样，在主体部分，您可能需要减少Inception模块的数量以及每个模块中的卷积层数量。对于头部，全局平均池化层和全连接层可能需要调整以适应较低分辨率的特征图。

3. 比较AlexNet、VGG、NiN和GoogLeNet的模型参数大小。后两个网络架构如何显著减少模型参数大小？

NiN和GoogLeNet通过以下方式显著减少了模型参数大小：NiN使用1x1卷积层来增加通道间的非线性，同时在网络末端使用全局平均池化来减少全连接层的需求。GoogLeNet通过Inception模块实现了多尺度特征学习，同时在应用较大卷积核之前使用1x1卷积层来减少通道数。这两种方法都有助于降低模型参数数量，从而减少计算成本和内存需求。

4. 比较GoogLeNet和AlexNet所需的计算量。这如何影响加速器芯片的设计，例如在内存大小、内存带宽、缓存大小、计算量以及专用操作的优势方面？

GoogLeNet相较于AlexNet具有更低的计算量。这对加速器芯片的设计产生了影响，例如：

- 内存大小：由于GoogLeNet具有较少的参数，因此可能需要较小的内存来存储模型权重。
- 内存带宽：由于较低的计算量，GoogLeNet可能需要较低的内存带宽。
- 缓存大小：GoogLeNet的较低计算量可能减少了对大型缓存的需求。
- 计算量：GoogLeNet需要较少的计算资源，这意味着加速器芯片可能需要较少的计算单元。
- 专用操作的优势：GoogLeNet中的1x1卷积和全局平均池化等操作可能使得专用硬件加速器在这些操作上具有更高的性能优势。

总之，GoogLeNet的较低计算量对加速器芯片的设计产生了影响，可能导致较小的内存大小、较低的内存带宽、较小的缓存大小、较少的计算单元和专用操作的优势。这些因素使得GoogLeNet在硬件加速器上的实现更加高效和可行，从而在实际应用中具有更好的性能